In [2]:
import cv2
import numpy as np
import pygame
from djitellopy import Tello
import time
from tflite_runtime.interpreter import Interpreter

# Parameters for object detection.
label_path_gates = "dict.txt"
model_path_gates = "model.tflite"
confidence_threshold_gate = 0.65

# Drone navigation parameters.
FORWARD_INC = 75
SPEED = 60

MAX_HEIGHT = 200

# Drone state and navigation directions.
_STATE_FLYING = "FLYING"
_NAV_FORWARD = "FORWARD"

# Pygame display settings.
SCREEN_WIDTH = 640
SCREEN_HEIGHT = 480

class Pilot:

    def __init__(self):
        # Initialize Pygame and display
        pygame.init()
        pygame.display.set_caption("Drone Camera Feed")
        self.screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
        self.clock = pygame.time.Clock()

        # Initialize Tello drone
        self.drone = Tello()
        self.state = _STATE_FLYING
        self.nav = ""

        # Initialize TensorFlow Lite interpreter for gate detection
        self.gates = Interpreter(model_path=model_path_gates)
        self.gates.allocate_tensors()
        self.gate_input = self.gates.get_input_details()
        self.gate_output = self.gates.get_output_details()

    def run(self):
        """Main loop running the drone."""
        self.drone.connect()
        self.drone.set_speed(SPEED)

        # self.drone.takeoff()
        print("Drone is launched")

        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.land()
                    return

            self.update()
            self.clock.tick(30)  # Cap FPS at 30

    def update(self):
      self.detect_gate()

      if self.nav == _NAV_FORWARD:
        # self.drone.move_forward(FORWARD_INC)
        time.sleep(2)  # Adjust as needed based on your scenario
        self.nav = ""

      # Check drone height and land if exceeds maximum height
      current_height = self.drone.get_height()
      if current_height > MAX_HEIGHT:
        print(f"Maximum height ({MAX_HEIGHT} cm) reached. Landing...")
        self.land()
        return

    # Display the drone's camera feed
      frame = self.drone.get_frame_read().frame
      frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
      frame = np.rot90(frame)  # Rotate the frame
      frame = pygame.surfarray.make_surface(frame)
      self.screen.blit(frame, (0, 0))
      pygame.display.update()



    def detect_gate(self):

        frame = self.drone.get_frame_read().frame
        frame_resized = cv2.resize(frame, (320, 320))
        input_data = np.expand_dims(frame_resized, axis=0)

        # Set input tensor for TensorFlow Lite model
        self.gates.set_tensor(self.gate_input[0]['index'], input_data)

        # Run inference
        self.gates.invoke()

        # Retrieve detection results
        boxes = self.gates.get_tensor(self.gate_output[0]['index'])[0]
        confidence = self.gates.get_tensor(self.gate_output[2]['index'])[0]

        # Check if a gate is detected based on confidence threshold
        if confidence > confidence_threshold_gate:
            self.nav = _NAV_FORWARD
            print("Gate detected, moving forward.")

    def land(self):
        self.drone.land()
        print("Drone has landed.")
        pygame.quit()

if __name__ == "__main__":
    pilot = Pilot()
    pilot.run()


pygame 2.6.0 (SDL 2.28.4, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


ImportError: dlopen(/Users/saumil/anaconda3/lib/python3.11/site-packages/tflite_runtime/_pywrap_tensorflow_interpreter_wrapper.so, 0x0002): tried: '/Users/saumil/anaconda3/lib/python3.11/site-packages/tflite_runtime/_pywrap_tensorflow_interpreter_wrapper.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/saumil/anaconda3/lib/python3.11/site-packages/tflite_runtime/_pywrap_tensorflow_interpreter_wrapper.so' (no such file), '/Users/saumil/anaconda3/lib/python3.11/site-packages/tflite_runtime/_pywrap_tensorflow_interpreter_wrapper.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64'))

This part was completely chatgpt-ed

In [4]:
import cv2
import numpy as np
import pygame
from djitellopy import Tello
import time
from tflite_runtime.interpreter import Interpreter

# Parameters for object detection.
label_path_gates = "dict.txt"
model_path_gates = "model.tflite"
confidence_threshold_gate = 0.65

# Drone navigation parameters.
FORWARD_INC = 75
SPEED = 60

MAX_HEIGHT = 200

# Drone state and navigation directions.
_STATE_FLYING = "FLYING"
_NAV_FORWARD = "FORWARD"
_NAV_LEFT = "LEFT"
_NAV_RIGHT = "RIGHT"
_NAV_UP = "UP"
_NAV_DOWN = "DOWN"

# Pygame display settings.
SCREEN_WIDTH = 640
SCREEN_HEIGHT = 480

class Pilot:

    def __init__(self):
        # Initialize Pygame and display
        pygame.init()
        pygame.display.set_caption("Drone Camera Feed")
        self.screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
        self.clock = pygame.time.Clock()

        # Initialize Tello drone
        self.drone = Tello()
        self.state = _STATE_FLYING
        self.nav = ""

        # Initialize TensorFlow Lite interpreter for gate detection
        self.gates = Interpreter(model_path=model_path_gates)
        self.gates.allocate_tensors()
        self.gate_input = self.gates.get_input_details()
        self.gate_output = self.gates.get_output_details()

    def run(self):
        """Main loop running the drone."""
        self.drone.connect()
        self.drone.set_speed(SPEED)

        self.drone.takeoff()
        print("Drone is launched")

        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.land()
                    return

            self.update()
            self.clock.tick(30)  # Cap FPS at 30

    def update(self):
        self.detect_gate()

        if self.nav == _NAV_FORWARD:
            self.drone.move_forward(FORWARD_INC)
            time.sleep(2)  # Adjust as needed based on your scenario
            self.nav = ""
        elif self.nav == _NAV_LEFT:
            self.drone.move_left(30)
            time.sleep(1)
            self.nav = _NAV_FORWARD
        elif self.nav == _NAV_RIGHT:
            self.drone.move_right(30)
            time.sleep(1)
            self.nav = _NAV_FORWARD
        elif self.nav == _NAV_UP:
            self.drone.move_up(30)
            time.sleep(1)
            self.nav = _NAV_FORWARD
        elif self.nav == _NAV_DOWN:
            self.drone.move_down(30)
            time.sleep(1)
            self.nav = _NAV_FORWARD

        # Check drone height and land if exceeds maximum height
        current_height = self.drone.get_height()
        if current_height > MAX_HEIGHT:
            print(f"Maximum height ({MAX_HEIGHT} cm) reached. Landing...")
            self.land()
            return

        # Display the drone's camera feed
        frame = self.drone.get_frame_read().frame
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
        frame = np.rot90(frame)  # Rotate the frame
        frame = pygame.surfarray.make_surface(frame)
        self.screen.blit(frame, (0, 0))
        pygame.display.update()

    def detect_gate(self):
        frame = self.drone.get_frame_read().frame
        frame_resized = cv2.resize(frame, (320, 320))
        input_data = np.expand_dims(frame_resized, axis=0)

        # Set input tensor for TensorFlow Lite model
        self.gates.set_tensor(self.gate_input[0]['index'], input_data)

        # Run inference
        self.gates.invoke()

        # Retrieve detection results
        boxes = self.gates.get_tensor(self.gate_output[0]['index'])[0]
        confidence = self.gates.get_tensor(self.gate_output[2]['index'])[0]

        # Check if a gate is detected based on confidence threshold
        if confidence > confidence_threshold_gate:
            self.nav = self.determine_navigation(boxes)
            print("Gate detected, moving:", self.nav)

    def determine_navigation(self, boxes):
        # Assuming boxes is a list of bounding boxes
        # Here we should determine the gate's position relative to the frame center
        # Adjust navigation based on the gate's position
        frame_center_x = SCREEN_WIDTH // 2
        frame_center_y = SCREEN_HEIGHT // 2

        gate_x_center = (boxes[1] + boxes[3]) / 2 * SCREEN_WIDTH
        gate_y_center = (boxes[0] + boxes[2]) / 2 * SCREEN_HEIGHT

        if abs(gate_x_center - frame_center_x) < 50 and abs(gate_y_center - frame_center_y) < 50:
            return _NAV_FORWARD
        elif gate_x_center < frame_center_x:
            return _NAV_LEFT
        elif gate_x_center > frame_center_x:
            return _NAV_RIGHT
        elif gate_y_center < frame_center_y:
            return _NAV_UP
        else:
            return _NAV_DOWN

    def land(self):
        self.drone.land()
        print("Drone has landed.")
        pygame.quit()

if __name__ == "__main__":
    pilot = Pilot()
    pilot.run()


ImportError: dlopen(/Users/saumil/anaconda3/lib/python3.11/site-packages/tflite_runtime/_pywrap_tensorflow_interpreter_wrapper.so, 0x0002): tried: '/Users/saumil/anaconda3/lib/python3.11/site-packages/tflite_runtime/_pywrap_tensorflow_interpreter_wrapper.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64')), '/System/Volumes/Preboot/Cryptexes/OS/Users/saumil/anaconda3/lib/python3.11/site-packages/tflite_runtime/_pywrap_tensorflow_interpreter_wrapper.so' (no such file), '/Users/saumil/anaconda3/lib/python3.11/site-packages/tflite_runtime/_pywrap_tensorflow_interpreter_wrapper.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64e' or 'arm64'))

In [ ]:
import cv2
import numpy as np
import pygame
from djitellopy import Tello
import time
from tflite_runtime.interpreter import Interpreter

# Parameters for object detection.
label_path_gates = "dict.txt"
model_path_gates = "model.tflite"
confidence_threshold_gate = 0.65

# Drone navigation parameters.
FORWARD_INC = 75
SPEED = 60

MAX_HEIGHT = 200

# Drone state and navigation directions.
_STATE_FLYING = "FLYING"
_NAV_FORWARD = "FORWARD"

# Pygame display settings.
SCREEN_WIDTH = 640
SCREEN_HEIGHT = 480

class Pilot:

    def __init__(self):
        # Initialize Pygame and display
        pygame.init()
        pygame.display.set_caption("Drone Camera Feed")
        self.screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
        self.clock = pygame.time.Clock()

        # Initialize Tello drone
        self.drone = Tello()
        self.state = _STATE_FLYING
        self.nav = ""

        # Initialize TensorFlow Lite interpreter for gate detection
        self.gates = Interpreter(model_path=model_path_gates)
        self.gates.allocate_tensors()
        self.gate_input = self.gates.get_input_details()
        self.gate_output = self.gates.get_output_details()

    def run(self):
        """Main loop running the drone."""
        self.drone.connect()
        self.drone.set_speed(SPEED)

        self.drone.takeoff()
        print("Drone is launched")

        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.land()
                    return

            self.update()
            self.clock.tick(30)  # Cap FPS at 30

    def update(self):
        self.detect_gate()

        if self.nav == _NAV_FORWARD:
            self.drone.move_forward(FORWARD_INC)
            time.sleep(2)  # Adjust as needed based on your scenario
            self.nav = ""

        # Check drone height and land if it exceeds the maximum height
        current_height = self.drone.get_height()
        if current_height > MAX_HEIGHT:
            print(f"Maximum height ({MAX_HEIGHT} cm) reached. Landing...")
            self.land()
            return

        # Display the drone's camera feed
        frame = self.drone.get_frame_read().frame
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
        frame = np.rot90(frame)  # Rotate the frame
        frame = pygame.surfarray.make_surface(frame)
        self.screen.blit(frame, (0, 0))
        pygame.display.update()

    def detect_gate(self):
        frame = self.drone.get_frame_read().frame
        frame_resized = cv2.resize(frame, (320, 320))
        input_data = np.expand_dims(frame_resized, axis=0).astype(np.float32)

        # Set input tensor for TensorFlow Lite model
        self.gates.set_tensor(self.gate_input[0]['index'], input_data)

        # Run inference
        self.gates.invoke()

        # Retrieve detection results
        boxes = self.gates.get_tensor(self.gate_output[0]['index'])[0]
        confidence = self.gates.get_tensor(self.gate_output[2]['index'])[0]

        # Check if a gate is detected based on confidence threshold
        if confidence > confidence_threshold_gate:
            self.nav = _NAV_FORWARD
            print("Gate detected, moving forward.")

    def land(self):
        self.drone.land()
        print("Drone has landed.")
        pygame.quit()

if __name__ == "__main__":
    pilot = Pilot()
    pilot.run()
